### Test of data instantiation

In [13]:
import torch
import numpy
from torch.utils.data import Dataset, DataLoader 
from torchvision.datasets import ImageFolder
import pandas as pd
from PIL import Image
import os


In [ ]:

        
# img = torch.ones(6, 3)
# labels = [0, 1, 1, 1, 0, 0]

# dataset = ExpDataSet(img, labels)
# print(dataset[4])



(tensor([1., 1., 1.]), tensor(0))


In [ ]:
# Local paths

label_path = "dataset/datasets/ringmerkingno/ringcodes.csv"

# Make sure to separate csv file on |
df = pd.read_csv(label_path, sep="|")

df.head()

,filename,code,color
0,LBNW(J22M)_20080310234907_79.jpg,J22M,NW
1,LBNW(J03Y)_20080311094828_79.jpg,J03Y,NW
2,LBBW(JR0T)_20080320193604_79.jpg,JR0T,BW
3,LBBW(J1EM)_20080321135128_79.jpg,J1EM,BW
4,LBBW(JZ3K)_20111027125715_79.jpg,JZ3K,BW


In [ ]:

dl = df.sort_values("filename", ascending=True).reset_index(drop=True)
dl.head()
exp_labels = dl[["code", "color"]]
exp_labels.head()


,filename,code,color
11531,RBWN(VA76)_20110428222633_385.jpg,VA76,WN
11532,RBWN(VA76)_20120330095526_79.jpg,VA76,WN
11533,RBWN(VA76)_20120611171008_385.jpg,VA76,WN
11534,RBWN(VE14)_20140314104129_385.jpg,VE14,WN
11535,RBYN(CR0A0)_20210816203135_764.jpg,CR0A0,YN


In [7]:
image_path = "dataset/datasets/ringmerkingno/images"
exp_x = ...



TypeError: ExpDataSet.__init__() missing 2 required positional arguments: 'images' and 'labels'

In [ ]:
class ExpDataSet(Dataset):
    def __init__(self, img_path:str, labels_path:str, transform=None):
        self.img_path = img_path
        self.transform = transform

        # Retrieve labels and sort them in alphabetical order to match the images
        df = pd.read_csv(labels_path, sep="|")
        self.df = df.sort_values("filename", ascending=True).reset_index(drop=True)
        
        # Retrieve images from folder and match them with labels
        self.img_paths = self.df["filename"].apply(lambda e: os.path.join(img_path, e))
        
        



    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        label = self.labels[idx]
        image = self.images[idx]

        if self.transform: # Add transformation mask to image
            image = self.transform(image)
        
        return image, label